# Excercise:

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.

https://platform.stratascratch.com/coding/10300-premium-vs-freemium?code_type=3&via=keith

Preliminary steps: load dataframes

In [2]:
import pandas as pd
import os

'data\\ms_acc_dimension.csv'

In [5]:
ms_acc_dimension =pd.read_csv(os.path.join('data','ms_acc_dimension.csv'),sep='\t')
ms_download_facts=pd.read_csv(os.path.join('data','ms_download_facts.csv'),sep='\t')
ms_user_dimension=pd.read_csv(os.path.join('data','ms_user_dimension.csv'),sep='\t')
ms_download_facts['date']=pd.to_datetime(ms_download_facts['date'])

In [4]:
ms_download_facts.sort_values('date')

,date,user_id,downloads
32,2020-08-15,33,6
71,2020-08-15,72,2
18,2020-08-15,19,6
15,2020-08-15,16,5
61,2020-08-15,62,6
...,...,...,...
34,2020-08-25,35,1
84,2020-08-25,85,7
43,2020-08-25,44,9
91,2020-08-25,92,7


## Execution: 

1) find which users are paying customers and which aren't by joining ms_acc_dimension and ms_user_dimension

2) plug resul into ms_download_facts

3) group by date, paying customer

In [5]:
paying_user= pd.merge(ms_acc_dimension, ms_user_dimension, how='inner')
paying_user

,acc_id,paying_customer,user_id
0,701,no,17
1,701,no,21
2,701,no,55
3,702,no,24
4,702,no,87
...,...,...,...
90,744,yes,89
91,744,yes,95
92,745,yes,33
93,745,yes,83


In [6]:
download_type=pd.merge(ms_download_facts,paying_user)[['date','paying_customer','downloads']]
download_type#['paying_customer']=download_type['paying_customer'].map({'yes':True, 'no':False})

,date,paying_customer,downloads
0,2020-08-24,no,6
1,2020-08-18,no,2
2,2020-08-24,yes,4
3,2020-08-19,yes,7
4,2020-08-21,no,3
...,...,...,...
90,2020-08-24,yes,3
91,2020-08-21,yes,8
92,2020-08-24,no,0
93,2020-08-21,yes,9


In [7]:
def paying(x):
    if x['paying_customer']=='yes':
        return x['downloads']
    else:
        return 0
def not_paying(x):
    if x['paying_customer']=='no':
        return x['downloads']
    else:
        return 0

In [8]:
download_type['paying']=download_type.apply(func= paying,axis=1)
download_type['non_paying']=download_type.apply(func= not_paying,axis=1)
download_type[['date','paying','non_paying']]

,date,paying,non_paying
0,2020-08-24,0,6
1,2020-08-18,0,2
2,2020-08-24,4,0
3,2020-08-19,7,0
4,2020-08-21,0,3
...,...,...,...
90,2020-08-24,3,0
91,2020-08-21,8,0
92,2020-08-24,0,0
93,2020-08-21,9,0


In [9]:
grouped=download_type.groupby('date').sum().reset_index()
grouped

,date,downloads,paying,non_paying
0,2020-08-15,30,19,11
1,2020-08-16,29,14,15
2,2020-08-17,54,9,45
3,2020-08-18,17,7,10
4,2020-08-19,26,13,13
5,2020-08-20,41,28,13
6,2020-08-21,49,17,32
7,2020-08-22,41,26,15
8,2020-08-23,35,23,12
9,2020-08-24,44,38,6


Now to only display the rows where not_paying > paying

In [10]:
grouped[grouped['non_paying']>grouped['paying']][['date','non_paying','paying']]

,date,non_paying,paying
1,2020-08-16,15,14
2,2020-08-17,45,9
3,2020-08-18,10,7
6,2020-08-21,32,17


## Other method: pivot tables

This kind of aggregation is easy to do with pivot tables:

In [13]:
aggregated=pd.pivot_table(download_type,index='date',columns='paying_customer',aggfunc='sum',values='downloads').reset_index()

    # df_pivot = pd.pivot_table(
    #     train, 
    #     index=f,
    #     columns="Transported",
    #     aggfunc='size'
print(aggregated[['date','yes','no']])

paying_customer       date  yes  no
0               2020-08-15   19  11
1               2020-08-16   14  15
2               2020-08-17    9  45
3               2020-08-18    7  10
4               2020-08-19   13  13
5               2020-08-20   28  13
6               2020-08-21   17  32
7               2020-08-22   26  15
8               2020-08-23   23  12
9               2020-08-24   38   6
10              2020-08-25   26  23


Last part: Include only records where non-paying customers have more downloads than paying customers

In [14]:

aggregated[aggregated['no']>aggregated['yes']]

paying_customer,date,no,yes
1,2020-08-16,15,14
2,2020-08-17,45,9
3,2020-08-18,10,7
6,2020-08-21,32,17
